In [ ]:

%pip install pandas openpyxl
%pip install transformers
%pip install newspaper3k
%pip install google-search-results
%pip install googletrans
%pip install llama-cpp-python
%pip install tqdm
%pip install googlesearch-python
%pip install seaborn
import pandas as pd
df_data = pd.read_excel("https://github.com/sentinel747/Applciation_AIB/raw/refs/heads/main/DataInputCRMExraction.xlsx")
df_context = pd.read_excel("https://github.com/sentinel747/Applciation_AIB/raw/refs/heads/main/ContextInputCategories.xlsx")
# Imposto le opzioni per mostrare tutte le righe e colonne
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


In [ ]:
import numpy as np
import requests
import pandas as pd
from tqdm import tqdm  # Importa tqdm per la barra di avanzamento

# Rimozione delle colonne inutili
df_data_cleaned = df_data.drop(columns=["Contact Name", "Contact Job Title"])

# Rimozione di duplicati e valori mancanti
df_data_cleaned = df_data_cleaned.drop_duplicates()
df_data_cleaned = df_data_cleaned.dropna()

# Estrazione del dominio dall'email
tqdm.pandas(desc="Estrazione domini")
df_data_cleaned['Email Domain'] = df_data_cleaned['Contact E-mail'].progress_apply(lambda email: email.split('@')[-1])

# Rimozione dei domini pubblici
public_domains = ['gmail.com', 'hotmail.com', 'yahoo.com', 'outlook.com', 'live.com', 'icloud.com', 'unknown.com']
df_data_cleaned = df_data_cleaned[~df_data_cleaned['Email Domain'].isin(public_domains)]

# Verifica dell'esistenza dei domini
def check_domain_exists(domain):
    try:
        response = requests.get(f"http://{domain}", timeout=2)
        return response.status_code == 200
    except requests.RequestException:
        return False

tqdm.pandas(desc="Verifica dei domini")
df_data_cleaned['Domain Exists'] = df_data_cleaned['Email Domain'].progress_apply(check_domain_exists)
df_data_cleaned = df_data_cleaned[df_data_cleaned['Domain Exists']]
df_data_cleaned = df_data_cleaned.drop(columns=['Domain Exists'])  # Rimuove la colonna temporanea

# Estrazione del nome principale del dominio (ad es., "savills" da "savills.es" e "savills.pt")
df_data_cleaned['Main Domain Name'] = df_data_cleaned['Email Domain'].apply(lambda domain: domain.split('.')[0])

# Rimozione dei duplicati basati sul nome principale del dominio mantenendo la prima occorrenza del dominio completo
df_data_cleaned = df_data_cleaned.drop_duplicates(subset=["Main Domain Name"])

# Rimozione della colonna temporanea "Main Domain Name"
df_data_cleaned = df_data_cleaned.drop(columns=['Main Domain Name'])

# Rimozione dei duplicati finali su "Contact E-mail" e "Email Domain"
df_data_cleaned = df_data_cleaned.drop_duplicates(subset=["Contact E-mail"])
df_data_cleaned = df_data_cleaned.drop_duplicates(subset=["Email Domain"])

# Salvataggio del risultato finale
df_data_cleaned.to_excel(r'C:\Users\Andrea\Downloads\df_data_cleaned.xlsx', index=False)

df_data_cleaned


In [ ]:
import numpy as np

# Funzione per pulire il dataset delle categorie
def clean_categories(df):
    # Rimuovi duplicati e valori nulli
    df_cleaned = df.drop_duplicates().dropna()

    # Rimuovi le colonne specificate
    df_cleaned = df_cleaned.drop(columns=['Can they buy the solution?', 'Can they influence the buying decision?'])

    return df_cleaned

# Applica la funzione di pulizia ai dati delle categorie
df_context_cleaned = clean_categories(df_context)
df_context_cleaned.to_excel(r'C:\Users\Andrea\Downloads\df_context_cleaned.xlsx', index=False)
df_context_cleaned.head()




In [ ]:
import requests
from bs4 import BeautifulSoup # type: ignore
import pandas as pd
from tqdm import tqdm  # Importa tqdm per la barra di avanzamento
from transformers import pipeline

# Inizializzazione del modello di Hugging Face per la traduzione
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-mul-en")  # Modello per tradurre in inglese

# Funzione per rimuovere duplicati da una lista di stringhe
def remove_duplicates(description_parts):
    seen = set()
    unique_parts = []
    for part in description_parts:
        if part not in seen:
            unique_parts.append(part)
            seen.add(part)
    return unique_parts

# Funzione migliorata per estrarre più informazioni dai meta tag e dai paragrafi <p>
def extract_meta_information(soup):
    info = {}

    # Estrarre meta description
    description = soup.find('meta', attrs={'name': 'description'})
    if description:
        info['meta_description'] = description.get('content', '')

    # Estrarre og:description (usato spesso per social e anteprime)
    og_description = soup.find('meta', attrs={'property': 'og:description'})
    if og_description:
        info['og_description'] = og_description.get('content', '')

    # Estrarre meta keywords (se disponibili)
    keywords = soup.find('meta', attrs={'name': 'keywords'})
    if keywords:
        info['meta_keywords'] = keywords.get('content', '')

    # Estrarre og:title (utilizzato per anteprime sui social)
    og_title = soup.find('meta', attrs={'property': 'og:title'})
    if og_title:
        info['og_title'] = og_title.get('content', '')

    # Estrazione di più paragrafi per raccogliere maggiori informazioni
    paragraphs = soup.find_all('p')
    if paragraphs:
        # Limitiamo l'estrazione a un massimo di 7 paragrafi, per evitare di estrarre troppo testo
        paragraph_text = " ".join([p.get_text().strip() for p in paragraphs[:7] if p.get_text().strip()])
        info['paragraphs'] = paragraph_text

    return info

# Funzione per estrarre informazioni dal sito web
def extract_company_info(url):
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')

            # Usa la funzione migliorata per estrarre le informazioni
            meta_info = extract_meta_information(soup)

            # Combina le informazioni raccolte per creare una descrizione
            description_parts = []

            # Aggiungi le varie descrizioni disponibili
            if 'meta_description' in meta_info:
                description_parts.append(meta_info['meta_description'])
            if 'og_description' in meta_info:
                description_parts.append(meta_info['og_description'])
            if 'paragraphs' in meta_info:
                description_parts.append(meta_info['paragraphs'])

            # Rimuovi duplicati dalle descrizioni
            unique_description_parts = remove_duplicates(description_parts)

            # Unisci le parti di descrizione
            full_description = " ".join(unique_description_parts)
            return full_description[:500]  # Limita a 500 caratteri
        else:
            return "Company information not available."
    except Exception as e:
        return "Company information not available."

# Funzione per tradurre la descrizione in inglese
def translate_to_english(text):
    try:
        translation = translator(text, max_length=500)
        return translation[0]['translation_text']
    except Exception as e:
        return text

# Funzione per creare il link del sito web a partire dal dominio email
def construct_website_url(domain):
    return f"http://{domain}"

# Funzione per aggiungere la colonna "Description" nel DataFrame con barra di progresso
def add_description_column(df):
    descriptions = []
    
    # Usa tqdm per visualizzare la barra di avanzamento
    for domain in tqdm(df['Email Domain'], desc="Estrazione informazioni", unit="azienda"):
        website_url = construct_website_url(domain)
        web_description = extract_company_info(website_url)
        translated_description = translate_to_english(web_description)
        descriptions.append(translated_description)

    df['Description'] = descriptions  # Aggiungi tutte le descrizioni al DataFrame
    return df

# Esegui la funzione per aggiungere la colonna 'Description' al DataFrame esistente
df_data_with_description = add_description_column(df_data_cleaned)
df_data_with_description.to_excel(r'C:\Users\Andrea\Downloads\df_data_with_company_description.xlsx', index=False)

df_data_with_description.head()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

# Funzione per eseguire la classificazione basata su TF-IDF e similarità coseno
def classify_with_cosine_similarity(df_data, df_context):
    # Combina le descrizioni e le note per costruire il vocabolario
    all_texts = df_data['Description'].tolist() + df_context['Notes'].tolist()

    # Creazione della rappresentazione TF-IDF per le descrizioni e le note
    vectorizer = TfidfVectorizer().fit(all_texts)
    company_vectors = vectorizer.transform(df_data['Description'])
    category_vectors = vectorizer.transform(df_context['Notes'])

    best_matches = []
    similarities_matrix = []

    # Calcola la similarità coseno tra ogni descrizione aziendale e le note delle categorie
    for i in tqdm(range(company_vectors.shape[0]), desc="Classificazione", unit="azienda"):
        similarities = cosine_similarity(company_vectors[i], category_vectors)
        similarities_matrix.append(similarities.flatten())  # Appiattisci la matrice delle similarità
        best_match_idx = similarities.argmax()  # Trova l'indice con la similarità massima
        best_category = df_context.iloc[best_match_idx]['Player']  # Ottieni la categoria corrispondente
        best_matches.append(best_category)

    # Assegna la migliore categoria trovata al DataFrame
    df_data['Best Category'] = best_matches
    return df_data, pd.DataFrame(similarities_matrix)

# Funzione per aggiungere la colonna "Best Category" e "Is Target" nel DataFrame
def add_classification_and_target_column(df_data, df_context):
    # Applica la classificazione basata su similarità coseno e ottieni la matrice di similarità
    df_data, similarities_matrix = classify_with_cosine_similarity(df_data, df_context)

    # Creazione della mappa per "Is Target"
    target_map = df_context.set_index('Player')['Is Target'].to_dict()

    # Funzione per verificare se la categoria è un target
    def is_target_category(category):
        return target_map.get(category, "No")  # Restituisce "No" se la categoria non è trovata

    # Aggiunge la colonna "Is Target" in base alla categoria assegnata
    df_data['Is Target'] = df_data['Best Category'].apply(lambda category: is_target_category(category))

    return df_data, similarities_matrix

# Funzione per visualizzare la distribuzione delle categorie (Target vs Non-Target)
def plot_target_distribution(df_data):
    plt.figure(figsize=(8, 6))
    ax = sns.countplot(x='Is Target', data=df_data)
    
    # Aggiungi le etichette sopra ogni barra
    for p in ax.patches:
        ax.annotate(f'{int(p.get_height())}', (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha='center', va='baseline', fontsize=12, color='black', xytext=(0, 5),
                    textcoords='offset points')

    plt.title('Distribuzione delle Aziende tra Target e Non-Target')
    plt.show()

# Funzione per visualizzare la distribuzione delle categorie (Player)
def plot_player_distribution(df_data):
    plt.figure(figsize=(12, 6))
    ax = sns.countplot(y='Best Category', data=df_data, order=df_data['Best Category'].value_counts().index)
    
    # Aggiungi le etichette per il numero di aziende per ciascuna categoria
    for p in ax.patches:
        ax.annotate(f'{int(p.get_width())}', (p.get_width(), p.get_y() + p.get_height() / 2),
                    ha='center', va='baseline', fontsize=12, color='black', xytext=(0, 5),
                    textcoords='offset points')

    plt.title('Distribuzione delle Aziende per Categoria (Player)')
    plt.show()

# Funzione per visualizzare la heatmap delle similarità coseno
def plot_similarity_heatmap(similarities_matrix):
    plt.figure(figsize=(10, 8))
    sns.heatmap(similarities_matrix, cmap="YlGnBu", xticklabels=False, yticklabels=False)
    plt.title("Heatmap delle Similarità Coseno tra Aziende e Categorie")
    plt.show()

# Funzione per visualizzare un istogramma della distribuzione delle similarità
def plot_similarity_histogram(similarities_matrix):
    plt.figure(figsize=(8, 6))
    sns.histplot(similarities_matrix.to_numpy().flatten(), bins=20, kde=True)  # Conversione in NumPy array
    plt.title("Distribuzione delle Similarità Coseno tra Aziende e Categorie")
    plt.xlabel("Valori di Similarità Coseno")
    plt.ylabel("Frequenza")
    plt.show()

# Funzione per visualizzare la percentuale di target
def plot_target_percentage(df_data):
    target_counts = df_data['Is Target'].value_counts(normalize=True) * 100
    plt.figure(figsize=(8, 6))
    plt.pie(target_counts, labels=target_counts.index, autopct='%1.1f%%', startangle=140, colors=['#ff9999','#66b3ff'])
    plt.title("Percentuale di Aziende Target vs Non-Target")
    plt.show()

# Applica la classificazione e la verifica del target utilizzando la similarità coseno
df_data_classified_cosinesimilarity, similarities_matrix_cosinesimilarity = add_classification_and_target_column(df_data_with_description, df_context_cleaned)

# Salva il DataFrame aggiornato con le categorie classificate e la colonna Is Target
output_path_llm = r'C:\Users\Andrea\Downloads\data_classified_cosinesimilarity.xlsx'
df_data_classified_cosinesimilarity.to_excel(output_path_llm, index=False)

# Visualizza i grafici
plot_target_distribution(df_data_classified_cosinesimilarity)
plot_target_percentage(df_data_classified_cosinesimilarity)  # Aggiunta del grafico per la percentuale di Target
plot_player_distribution(df_data_classified_cosinesimilarity)
plot_similarity_heatmap(similarities_matrix_cosinesimilarity)
plot_similarity_histogram(similarities_matrix_cosinesimilarity)

df_data_classified_cosinesimilarity
